In [1]:
# import essential data handling libraries
import pandas as pd
import numpy as np
import json
import requests
from geopy.distance import geodesic
import time
from datetime import datetime

In [2]:
def get_hdb_resale_data():
    # importing our data set from data.gov.sg
    query_string='https://data.gov.sg/api/action/datastore_search?resource_id=42ff9cfe-abe5-4b54-beda-c88f9bb438ee&limit=1000000'
    resp = requests.get(query_string)

    #Convert JSON into Python Object 
    data = json.loads(resp.content)

    # Checking the length of dataframeenrich_geo_coordinate
    len(data['result']['records']) 

    # Store our dictionart records into hdb_price_dict_records
    hdb_price_dict_records = data['result']['records']

    # Next we need to feed our JSON data into dataframe. 
    # We will access the 'records' key:value pairs of the python dictionary. 
    # We will then map the list into a dataframe.
    town = []
    flat_type = []
    flat_model = []
    floor_area_sqm = []
    street_name = []
    resale_price = []
    month = []
    remaining_lease = []
    lease_commence_date = []
    storey_range = []
    _id = []
    block = []

    for i in range(0, len(hdb_price_dict_records)):
        town.append(hdb_price_dict_records[i]['town'])
        flat_type.append(hdb_price_dict_records[i]['flat_type'])
        flat_model.append(hdb_price_dict_records[i]['flat_model'])
        floor_area_sqm.append(hdb_price_dict_records[i]['floor_area_sqm'])
        street_name.append(hdb_price_dict_records[i]['street_name'])
        resale_price.append(hdb_price_dict_records[i]['resale_price'])
        month.append(hdb_price_dict_records[i]['month'])
        remaining_lease.append(hdb_price_dict_records[i]['remaining_lease'])
        lease_commence_date.append(hdb_price_dict_records[i]['lease_commence_date'])
        storey_range.append(hdb_price_dict_records[i]['storey_range'])
        _id.append(hdb_price_dict_records[i]['_id'])
        block.append(hdb_price_dict_records[i]['block'])
        
    df = pd.DataFrame({
        'town': town,
        'flat_type': flat_type,
        'flat_model': flat_model,
        'floor_area_sqm': floor_area_sqm,
        'street_name': street_name,
        'resale_price': resale_price,
        'month': month,
        'remaining_lease': remaining_lease,
        'lease_commence_date': lease_commence_date,
        'storey_range': storey_range,
        '_id': _id,
        'block': block
    })

    df['address'] = df['block'] + ' ' + df['street_name']

    return df

In [3]:
df_hdb_price = get_hdb_resale_data()

In [4]:
# Let's examine our dataframe
df_hdb_price.head()

,town,flat_type,flat_model,floor_area_sqm,street_name,resale_price,month,remaining_lease,lease_commence_date,storey_range,_id,block,address
0,ANG MO KIO,2 ROOM,Improved,44,ANG MO KIO AVE 10,232000,2017-01,61 years 04 months,1979,10 TO 12,1,406,406 ANG MO KIO AVE 10
1,ANG MO KIO,3 ROOM,New Generation,67,ANG MO KIO AVE 4,250000,2017-01,60 years 07 months,1978,01 TO 03,2,108,108 ANG MO KIO AVE 4
2,ANG MO KIO,3 ROOM,New Generation,67,ANG MO KIO AVE 5,262000,2017-01,62 years 05 months,1980,01 TO 03,3,602,602 ANG MO KIO AVE 5
3,ANG MO KIO,3 ROOM,New Generation,68,ANG MO KIO AVE 10,265000,2017-01,62 years 01 month,1980,04 TO 06,4,465,465 ANG MO KIO AVE 10
4,ANG MO KIO,3 ROOM,New Generation,67,ANG MO KIO AVE 5,265000,2017-01,62 years 05 months,1980,01 TO 03,5,601,601 ANG MO KIO AVE 5


In [5]:
df_hdb_price.shape

(70104, 13)

In [6]:
# Create a function that creates data frames of key locations
def get_geo_coordinates(df):
    list_of_location = df.iloc[:,1].tolist()
    # Obtaining Mall Coordinates in Singapore
    address_name = []
    address_roadname = []
    address_lat = []
    address_long = []

    for i in range(0, len(list_of_location)):
        query_address = list_of_location[i]
        query_string = 'https://developers.onemap.sg/commonapi/search?searchVal='+str(query_address)+'&returnGeom=Y&getAddrDetails=Y'
        resp = requests.get(query_string)
        data = json.loads(resp.content)
        
        if data['found'] != 0:
            address_name.append(query_address)
            address_roadname.append(data["results"][0]["ROAD_NAME"])
            address_lat.append(data["results"][0]["LATITUDE"])
            address_long.append(data["results"][0]["LONGITUDE"])

            print (str(query_address)+ " ,Lat: " +data['results'][0]['LATITUDE'] + " Long: " + data['results'][0]['LONGITUDE'])

        else:
            print ("No Results")
            
    # Store this information in a dataframe
    return pd.DataFrame({
                'address': address_name,
                'roadName': address_roadname,
                'latitude': address_lat,
                'longitude': address_long
            })

In [11]:
# import csv of locations of interests
df_shopping_malls_list = pd.read_csv("./shopping_malls.csv")
df_mrt_stations_list = pd.read_csv("./data_sets/mrt_stations.csv")
df_govt_funded_schools_list = pd.read_csv("./data_sets/govt_funded_schools.csv")
df_govt_aided_schools_list = pd.read_csv("./data_sets/govt_aided_schools.csv")
df_sap_schools_list = pd.read_csv("./data_sets/sap_schools.csv")

df_mall = get_geo_coordinates(df_shopping_malls_list)
df_mrt = get_geo_coordinates(df_mrt_stations_list)
df_govt_funded_schools = get_geo_coordinates(df_govt_funded_schools_list)
df_govt_aided_schools = get_geo_coordinates(df_govt_aided_schools_list)
df_sap_schools = get_geo_coordinates(df_sap_schools_list)

df_mall.to_csv('./data_sets/df_mall.csv', index = False)
df_mrt.to_csv('./data_sets/df_mrt.csv', index = False)
df_govt_funded_schools.to_csv('./data_sets/df_govt_funded_schools.csv', index = False)
df_govt_aided_schools.to_csv('./data_sets/df_govt_aided_schools.csv', index = False)
df_sap_schools.to_csv('./data_sets/df_sap_schools.csv', index = False)


100 AM ,Lat: 1.2745882179999999 Long: 103.84347070000001
313@Somerset ,Lat: 1.301385102 Long: 103.8376844
Aperia ,Lat: 1.30974242 Long: 103.8641016
Balestier Hill Shopping Centre ,Lat: 1.326124169 Long: 103.8437095
Bugis Cube ,Lat: 1.298195005 Long: 103.85565550000001
Bugis Junction ,Lat: 1.300117893 Long: 103.8561916
Bugis+ ,Lat: 1.300779548 Long: 103.85632779999999
Capitol Piazza ,Lat: 1.293063359 Long: 103.85129350000001
Cathay Cineleisure Orchard ,Lat: 1.301497434 Long: 103.8364448
City Gate ,Lat: 1.302347543 Long: 103.86225999999999
City Square Mall ,Lat: 1.31147745 Long: 103.8567808
CityLink Mall ,Lat: 1.292378607 Long: 103.854663
No Results
Duo ,Lat: 1.299216621 Long: 103.8579968
Far East Plaza ,Lat: 1.3073111259999999 Long: 103.83394
Funan ,Lat: 1.2912440059999999 Long: 103.8499848
Great World City ,Lat: 1.293154929 Long: 103.831928
HDB Hub ,Lat: 1.33218812 Long: 103.8470778
No Results
ION Orchard ,Lat: 1.3040425340000001 Long: 103.83188829999999
Junction 8 ,Lat: 1.350239430999

KeyboardInterrupt: 

In [12]:
df_mall.to_csv('./data_sets/df_mall.csv', index = False)
df_mrt.to_csv('./data_sets/df_mrt.csv', index = False)
df_govt_funded_schools.to_csv('./data_sets/df_govt_funded_schools.csv', index = False)
df_govt_aided_schools.to_csv('./data_sets/df_govt_aided_schools.csv', index = False)
df_sap_schools.to_csv('./data_sets/df_sap_schools.csv', index = False)

In [8]:
def enrich_lat_lg_unique(df):
    # Create address column
    if 'address' not in df.columns:
        df['address'] = df['block'] + " " + df['street_name']
    
    # Dedup Address List
    df_dedup = df.drop_duplicates(subset='address', keep='first')
    df_dedup['address'] = df_dedup['address'].str.replace("ST. GEORGE'S", "SAINT GEORGE'S")

    # Next let's grab the unique addresses and create a list 
    address_list = df_dedup['address'].tolist()

    no_items = len(address_list)
    latitude = []
    longitude = []
    blk_no = []
    road_name = []
    postal_code = []
    address = []
    count = 0
    print(f"{no_items} rows of addresses to enrich")
    for row in range(len(address_list)):
        #formulate query string  
        query_address = address_list[row]
        query_string='https://developers.onemap.sg/commonapi/search?searchVal='+str(query_address)+'&returnGeom=Y&getAddrDetails=Y'
        resp = requests.get(query_string)

        #Convert JSON into Python Object 
        data_geo_location=json.loads(resp.content)

        if data_geo_location['found'] != 0:
            latitude.append(data_geo_location['results'][0]['LATITUDE'])
            longitude.append(data_geo_location['results'][0]['LONGITUDE'])
            blk_no.append(data_geo_location['results'][0]['BLK_NO'])
            road_name.append(data_geo_location['results'][0]['ROAD_NAME'])
            postal_code.append(data_geo_location['results'][0]['POSTAL'])
            address.append(query_address)
            print (str(round((row / no_items)*100, 2)) + "% " + "Completed: " + str(query_address) + " ,Lat: " + data_geo_location['results'][0]['LATITUDE'] + " Long: " + data_geo_location['results'][0]['LONGITUDE'])
        else:
            latitude.append(None)
            longitude.append(None)
            blk_no.append(None)
            road_name.append(None)
            postal_code.append(None)
            address.append(query_address)
            print (str(round((row / no_items)*100, 2)) + "%" + ": " + str(query_address) +  "No Results")

    # Fit into a dataframe
    return pd.DataFrame({
                'latitude': latitude,
                'longitude': longitude,
                'blk_no': blk_no,
                'road_name': road_name,
                'postal_code': postal_code,
                'address': address
            })

In [9]:
start_time = time.time()
df_enriched = enrich_lat_lg_unique(df_hdb_price)
print("--- %s seconds ---" % (time.time() - start_time))

8712 rows of addresses to enrich
0.0% Completed: 406 ANG MO KIO AVE 10 ,Lat: 1.362004539 Long: 103.85387990000001
0.01% Completed: 108 ANG MO KIO AVE 4 ,Lat: 1.370966352 Long: 103.83820190000002
0.02% Completed: 602 ANG MO KIO AVE 5 ,Lat: 1.3809284309999998 Long: 103.8359538
0.03% Completed: 465 ANG MO KIO AVE 10 ,Lat: 1.3662010409999998 Long: 103.857201
0.05% Completed: 601 ANG MO KIO AVE 5 ,Lat: 1.381041355 Long: 103.8351317
0.06% Completed: 150 ANG MO KIO AVE 5 ,Lat: 1.376806785 Long: 103.84201800000001


KeyboardInterrupt: 

In [10]:
df_enriched.shape

(8712, 6)

In [11]:
def export_missing_values(df):
    if df['latitude'].isna().sum() != 0:
        df[df['latitude'].isna()].to_csv("./data_sets/missing_lat_lng"+datetime.today().strftime("%Y-%m-%d")+".csv", index = False)
        df_missing_lat_lng.dropna().to_csv("./data_sets/unique_address_lat_lng_with_missing"+datetime.today().strftime("%Y-%m-%d")+".csv", index = False)
    else:
        df.to_csv("./data_sets/unique_address_lat_lng_no_missing"+datetime.today().strftime("%Y-%m-%d")+".csv", index = False)

In [12]:
export_missing_values(df_enriched)

In [13]:
df_enriched.dropna(inplace=True)
df_enriched.isna().sum()

latitude       0
longitude      0
blk_no         0
road_name      0
postal_code    0
address        0
dtype: int64

In [14]:
def get_lat_lng(df):
    list_of_lat = df['latitude']
    list_of_lng = df['longitude']
    list_of_lat_lng = []
    for lat, lng in zip(list_of_lat, list_of_lng):
        list_of_lat_lng.append((lat,lng))
    return list_of_lat_lng

In [15]:
hdb_lat_lng = get_lat_lng(df_enriched)
mall_lat_lng = get_lat_lng(df_mall)
mrt_lat_lng = get_lat_lng(df_mrt)
govt_aided_lat_lng = get_lat_lng(df_govt_aided_schools)
govt_funded_lat_lng = get_lat_lng(df_govt_funded_schools)
sap_lat_lng = get_lat_lng(df_sap_schools)

In [16]:
def get_nearest_dist_m(lst_origin, lst_destination):
    list_of_distances = []
    shortest_distance = []

    for origin in lst_origin:
        for destination in range(0, len(lst_destination)):
          list_of_distances.append(geodesic(origin, lst_destination[destination]).meters)
        shortest = min(list_of_distances)
        print(f'Shortest distance is {shortest} meters')
        shortest_distance.append(shortest)
        list_of_distances.clear()
    return shortest_distance

In [17]:
start_time = time.time()
nearest_mrt_dist = get_nearest_dist_m(hdb_lat_lng, mrt_lat_lng)
nearest_mall_dist = get_nearest_dist_m(hdb_lat_lng, mall_lat_lng)
nearest_govt_aided_dist = get_nearest_dist_m(hdb_lat_lng, govt_aided_lat_lng)
nearest_govt_funded_dist = get_nearest_dist_m(hdb_lat_lng, govt_funded_lat_lng)
nearest_sap_dist = get_nearest_dist_m(hdb_lat_lng, sap_lat_lng)
print("--- %s seconds ---" % (time.time() - start_time))

ce is 8868.898902867026 meters
Shortest distance is 9106.762193145029 meters
Shortest distance is 8979.610395977472 meters
Shortest distance is 8085.723291567563 meters
Shortest distance is 9238.33223063257 meters
Shortest distance is 5682.6973577556755 meters
Shortest distance is 6555.378212911782 meters
Shortest distance is 5505.920591157376 meters
Shortest distance is 6500.848159000282 meters
Shortest distance is 4760.0393139766875 meters
Shortest distance is 6545.332098779419 meters
Shortest distance is 6547.645991209826 meters
Shortest distance is 6624.718213951538 meters
Shortest distance is 1852.9097101927698 meters
Shortest distance is 1656.2847649256414 meters
Shortest distance is 343.98608706142966 meters
Shortest distance is 1139.1160079656959 meters
Shortest distance is 4301.299195652244 meters
Shortest distance is 3560.855823572457 meters
Shortest distance is 3921.4882960575587 meters
Shortest distance is 5677.625644203869 meters
Shortest distance is 5571.208246159869 mete

In [18]:
def get_cbd_dist_m(lst_origin):
    cbd_dist = []
    for origin in lst_origin:
        cbd_dist.append(geodesic(origin,(1.2830, 103.8513)).meters)


In [19]:
cbd_dist = get_cbd_dist_m(hdb_lat_lng)

In [20]:
# Creating new columns in dataframe
df_enriched['nearest_mrt_dist'] = nearest_mrt_dist
df_enriched['nearest_mall_dist'] = nearest_mall_dist
df_enriched['nearest_govt_aided_dist'] = nearest_govt_aided_dist
df_enriched['nearest_funded_dist'] = nearest_govt_funded_dist
df_enriched['nearest_sap_dist'] = nearest_sap_dist

In [21]:
# Re-merging dataframes
combined = df_enriched.merge(df_hdb_price, on="address", how="outer")

In [22]:
combined.shape

(70126, 23)

In [23]:
combined.dropna(inplace=True)

In [24]:
combined.drop(['blk_no', 'road_name'], axis=1, inplace=True)

In [25]:
combined.to_csv("./data_sets/combined_"+datetime.today().strftime("%Y-%m-%d")+".csv", index = False)